# Partido Diablos vs Sultanes 2025-06-17

Notebook para analizar y generar reportes del partido entre Diablos Rojos y Sultanes de Monterrey del 17 de junio de 2025. Se utilizan datos de TrackMan y la API de MLB.

* [TrackMan](https://support.trackmanbaseball.com/hc/en-us/articles/5089413493787-V3-FAQs-Radar-Measurement-Glossary-Of-Terms)
* [MLB](https://github.com/MajorLeagueBaseball/google-cloud-mlb-hackathon/tree/main/datasets/mlb-statsapi-docs)

## Importar bibliotecas y datos

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ruta = '../data/20250617-EstadioAlfredo-1.csv'
df = pd.read_csv(ruta, sep=',', encoding='utf-8')
df.head()

## Exploracion inicial de datos

In [ ]:
df.shape

In [ ]:
list(df.columns)

In [ ]:
df.dtypes

In [ ]:
df.describe()

## Limpiar datos

In [ ]:
# Elimiar columnas que no se van a usar
df = df.drop(columns=[
    'UTCDate', 'UTCTime', 'LocalDateTime', 'UTCDateTime', 'PitcherId', 'PitcherTeam', 'BatterId',
    'BatterTeam', 'PitcherSet', 'AutoPitchType', 'AutoHitType', 'Notes', 'Distance', 'Stadium',
    'HomeTeamForeignID', 'AwayTeamForeignID', 'GameForeignID', 'Level', 'League', 'GameID',
    'PitchUID', 'System'
  ], axis=1)

In [ ]:
df['Pitcher'] = df['Pitcher'].str.split(', ').apply(lambda x: x[1] + ' ' + x[0])
df['Batter'] = df['Batter'].str.split(', ').apply(lambda x: x[1] + ' ' + x[0])

## Separar en 2 dataframes uno para local y otro para visitante

In [ ]:
homeBatting = df[df['Top/Bottom'] == 'Bottom'].copy()
awayBatting = df[df['Top/Bottom'] == 'Top'].copy()

## Obtener el lineup de cada equipo

In [ ]:
print('Diablos Rojos: ', homeBatting['Batter'].unique().tolist()[:9])
print('Sultanes de Monterrey: ', awayBatting['Batter'].unique().tolist()[:9])

## Obtener los pitchers de cada equipo

In [ ]:
print('Diablos Rojos: ', awayBatting['Pitcher'].unique().tolist())
print('Sultanes de Monterrey: ', homeBatting['Pitcher'].unique().tolist())

## Análisis de pitchers

### Diablos

Generar una tabla con la informacion general de Wilmert Font, entrada en la que entro, entradas lanzadas, numero de lanzamientos, strikeouts, carreras

Generar tabla con la informacion de Wilmert Font, incluir tipo de lanzamiento, numero de veces lanzado, %uso, velocidad maxima, velocidad promedio, % de strikes, % swing strike, % de bolas, % de strikeout y % de bases por bolas

In [ ]:
fontDf = awayBatting[awayBatting['Pitcher'] == 'Font, Wilmer'][['TaggedPitchType', 'PitchCall', 'KorBB', 'RelSpeed']].copy()
fontDf = pd.get_dummies(fontDf, columns=['PitchCall', 'KorBB'], dtype=int)
fontDf['strike'] = fontDf['PitchCall_StrikeCalled'] + fontDf['PitchCall_StrikeSwinging']
fontDf['ball'] = fontDf['PitchCall_BallCalled'] + fontDf['PitchCall_BallinDirt']
fontDf.drop(columns=['PitchCall_StrikeCalled', 'PitchCall_BallCalled', 'PitchCall_BallinDirt', 'KorBB_Undefined'], inplace=True)
fontResults = fontDf.groupby('TaggedPitchType').agg(
    Usage=('TaggedPitchType', 'count'),
    UsagePct=('TaggedPitchType', lambda x: round(len(x) / len(fontDf) * 100, 1)),
    MaxSpeed=('RelSpeed', lambda x: round(x.max(), 1)),
    AvgSpeed=('RelSpeed', lambda x: round(x.mean(), 1)),
    strikePct=('strike', lambda x: round(sum(x) / len(x) * 100, 1)),
    swingStrikePct=('PitchCall_StrikeSwinging', lambda x: round(sum(x) / len(x) * 100, 1)),
    ballPct=('ball', lambda x: round(sum(x) / len(x) * 100, 1)),
    stikeOutPct=('KorBB_Strikeout', lambda x: round(sum(x) / sum(fontDf['KorBB_Strikeout']) * 100, 1)),
    walkPct=('KorBB_Walk', lambda x: round(sum(x) / sum(fontDf['KorBB_Walk']) * 100, 1))
).sort_values(by='Usage', ascending=False).reset_index()
fontResults

In [ ]:
def printBox():
    fig = plt.figure(figsize=(12, 8))
    fig.ad
    return fig
printBox().show()